In [ ]:
import rasterio
import numpy as np
import xarray as xr
import hvplot.xarray
import holoviews as hv
import geoviews as gv
#import rioxarray as rx
import os
import panel as pn
from holoviews import opts
hv.extension('bokeh', width=90)
pn.extension()


In [ ]:
os.environ["AWS_REQUEST_PAYER"] = "requester" 

# Basic idea with map and max and min height

In [ ]:
places={}
places['St.Thomas']="https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n19w065/USGS_1_n19w065.tif"
places['St.Croix']="https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n18w065/USGS_1_n18w065.tif"
places['Cornell']='https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n42w076/USGS_1_n42w076.tif'

In [ ]:
data = xr.open_rasterio(places['St.Thomas'])

In [ ]:
maxheight = float(data.where(data>0).max())
minheight = float(data.where(data>0).min())

In [ ]:
css = '''
.panel-widget-box {
  background: #f0f0f0;
  border-radius: 5px;
  border: 1px black solid;
}
'''
pn.extension(raw_css=[css])

In [ ]:
title = pn.pane.Markdown('# Visualizing Graphical Data')
greatest = pn.pane.Markdown('# Max height is '+ str(maxheight))
least = pn.pane.Markdown('# The Min height is '+ str(minheight))

In [ ]:
def options(Where):
    data = xr.open_rasterio(Where)
    data = data.where(data>0)[0,:,:]
    maxheight = round(float(data.where(data>0).max()))
    minheight = round(float(data.where(data>0).min()))
    greatest.object = '# Max height: '+ str(maxheight)
    least.object = '# Min height: '+ str(minheight)
    return gv.tile_sources.OSM*data.hvplot.quadmesh(rasterize=True,geo=True,colormap='viridis',project = True, tiles = 'OSM')

In [ ]:
pn.Column(
    title,
    pn.Row(
        pn.Column(pn.interact(options, Where=places),css_classes=['panel-widget-box'],margin=25),
        pn.Column(greatest,least,css_classes=['panel-widget-box'],margin=25),
        margin = (50,100),
        background='#e3c3ee',
        css_classes=['panel-widget-box']
    ),
    margin = 100,
    css_classes=['panel-widget-box'],
    background='#f96080'
)

# Try to bring in xarray-spatial

In [ ]:
import xrspatial as xrs
from datashader.colors import Elevation
from datashader.transfer_functions import shade

In [ ]:
hvplot.help('image', docstring=False, generic=False)

In [ ]:
forxr = data.where(data>0)[0,:,:]

In [ ]:
hillshaded = xrs.hillshade(forxr)

In [ ]:
slates = xrs.quantile(forxr,k=100,ignore_vals=(0,))

In [ ]:
#hillshaded.hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = 'kg')
slates.hvplot.image(rasterize = True, geo = True, tiles = 'OSM', cmap = 'inferno')

In [ ]:
#